<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/0_Ejercicio_Anonimizaci%C3%B3n_Fichas_Programacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de Fichas de Programación

En este script extraemos la programación de un conjunto de profesionale de un servicio clínico

In [155]:
import pandas as pd
import openpyxl as xl
import requests
import os

# Localización de los datos
path_fichas="https://raw.githubusercontent.com/rlagosb/taller_eiv/main/data/programacion/Fichas/"
planilla_fichas = 'Fichas Unidad Endoscopia.xlsx'
# Carpeta de destino
datos_crudos = '/content/datos_crudos/'
if not os.path.exists(datos_crudos): os.makedirs(datos_crudos)

In [158]:
#@title Descargar Planilla

# Descargar el archivo
response = requests.get(path_fichas + planilla_fichas)
with open(planilla_fichas, 'wb') as f:
    f.write(response.content)

# Cargar planilla
wb = xl.load_workbook(filename = planilla_fichas, data_only=True)

print(f'Planilla {planilla_fichas} descargada ✅')

Planilla Fichas Unidad Endoscopia.xlsx descargada ✅


In [159]:
#@title Obtener fichas

# Cargar datos de las fichas en las hojas de la planilla
fichas = {}
print('Cargando fichas:')

for ws_name in [name for name in wb.sheetnames if name not in ['Personal', 'Prestaciones', 'Ficha Programación']]:

  #Cargar hoja
  ws=wb[ws_name]

  #obtener datos profesional y eliminar cabecera
  profesional={'rut':ws['C4'].value,'nombre':ws['C3'].value,'profesion':ws['C5'].value,'obs_ficha':ws['F5'].value}
  ws.delete_rows(1,9)
  ws.delete_cols(1,1)

  #obtener actividades y horarios
  prog = (pd.DataFrame(ws.values).rename(columns={0:"ini",1:"fin"}).
          melt(id_vars=['ini','fin'], var_name='dia', value_name='act').
          # eliminar bloques sin actividades programadas
          dropna(subset=['act']))
  prog['dia']=prog['dia']-1

  #agregar datos profesional
  for col in ['rut','nombre','profesion','obs_ficha']:
    prog[col]=profesional[col]

  #agrupar bloques de actividades
  prog['id_prog']=((prog.rut!=prog.rut.shift()) |             # agrupar por cambio de rut
                    (prog.dia!=prog.dia.shift()) |            # por cambio de dia
                    (prog.act!=prog.act.shift())              # por cambio de actividad
                    ).cumsum()                                # asignar correlativo
  fichas[ws_name]=prog.groupby(['id_prog','rut','nombre','profesion','dia','act','obs_ficha'],as_index=False, dropna=False).aggregate({'ini':'min','fin':'max'})

  print(ws.title, '✔')



Cargando fichas:
OF ✔
JH ✔
VR ✔
EG ✔
VJ ✔
JG ✔
MG ✔
LR ✔
DR ✔


In [160]:
#@title Tabular datos

#Consolidar  fichas en una tabla
programacion=pd.concat(fichas, ignore_index=True)

# Calcular duración actividades
programacion['hrs_prog']=(pd.to_datetime(programacion['fin'].astype(str), format='%H:%M:%S') -
                     pd.to_datetime(programacion['ini'].astype(str), format='%H:%M:%S')).dt.total_seconds() / 3600

#Exportar a pm_programacion_medica
programacion.to_excel(datos_crudos+"Programacion.xlsx",sheet_name="PM")
print(programacion.info())
programacion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_prog    133 non-null    int64  
 1   rut        133 non-null    int64  
 2   nombre     133 non-null    object 
 3   profesion  133 non-null    object 
 4   dia        133 non-null    object 
 5   act        133 non-null    object 
 6   obs_ficha  27 non-null     object 
 7   ini        133 non-null    object 
 8   fin        133 non-null    object 
 9   hrs_prog   133 non-null    float64
dtypes: float64(1), int64(2), object(7)
memory usage: 10.5+ KB
None


,id_prog,rut,nombre,profesion,dia,act,obs_ficha,ini,fin,hrs_prog
0,1,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Consulta nueva,Comisión de Servicio,08:00:00,10:00:00,2.0
1,2,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Otras Actividades Clínicas,Comisión de Servicio,10:00:00,14:00:00,4.0
2,3,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Colonoscopía,Comisión de Servicio,14:00:00,15:00:00,1.0
3,4,123456,Orlando Flores,Médico(a) Cirujano(a),1,[GA] Endoscopía digestiva,Comisión de Servicio,15:00:00,16:00:00,1.0
4,5,123456,Orlando Flores,Médico(a) Cirujano(a),2,[GA] Otras Actividades Clínicas,Comisión de Servicio,08:00:00,09:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...
128,12,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,[GA] Comité Clínico,Encargada Unidad,08:30:00,10:30:00,2.0
129,13,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,[GA] Otras Actividades Clínicas,Encargada Unidad,10:30:00,13:00:00,2.5
130,14,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,Colación,Encargada Unidad,13:00:00,13:30:00,0.5
131,15,369874,Daniela Ruiz,Médico(a) Cirujano(a),5,[GA] Otras Actividades Clínicas,Encargada Unidad,13:30:00,14:00:00,0.5


# Pseudo-anonimización
Eliminamos los datos personales y reemplazamos el identificador

In [167]:
# Cremos un identificador alternativo

profesionales = programacion[['rut']].drop_duplicates(ignore_index=True).reset_index().rename(columns={'index':'idProfesional'})
profesionales['idProfesional'] +=1
profesionales

,idProfesional,rut
0,1,123456
1,2,678910
2,3,321654
3,4,987321
4,5,951843
5,6,357681
6,7,852963
7,8,147258
8,9,369874


In [171]:
# Agregamos el pseudoidentificador
# y eliminamos los datos personales

programacion.merge(profesionales, on='rut', how='left').drop(columns=['rut','nombre','profesion'])

,id_prog,dia,act,obs_ficha,ini,fin,hrs_prog,idProfesional
0,1,1,[GA] Consulta nueva,Comisión de Servicio,08:00:00,10:00:00,2.0,1
1,2,1,[GA] Otras Actividades Clínicas,Comisión de Servicio,10:00:00,14:00:00,4.0,1
2,3,1,[GA] Colonoscopía,Comisión de Servicio,14:00:00,15:00:00,1.0,1
3,4,1,[GA] Endoscopía digestiva,Comisión de Servicio,15:00:00,16:00:00,1.0,1
4,5,2,[GA] Otras Actividades Clínicas,Comisión de Servicio,08:00:00,09:00:00,1.0,1
...,...,...,...,...,...,...,...,...
128,12,5,[GA] Comité Clínico,Encargada Unidad,08:30:00,10:30:00,2.0,9
129,13,5,[GA] Otras Actividades Clínicas,Encargada Unidad,10:30:00,13:00:00,2.5,9
130,14,5,Colación,Encargada Unidad,13:00:00,13:30:00,0.5,9
131,15,5,[GA] Otras Actividades Clínicas,Encargada Unidad,13:30:00,14:00:00,0.5,9


In [172]:
# Datos Anonimizados

programacion.groupby(['profesion','act'], dropna=False).hrs_prog.sum().reset_index()



,profesion,act,hrs_prog
0,Médico(a) Cirujano(a),Colación,5.0
1,Médico(a) Cirujano(a),[GA] Colonoscopía,29.5
2,Médico(a) Cirujano(a),[GA] Comité Clínico,15.5
3,Médico(a) Cirujano(a),[GA] Consulta control,1.0
4,Médico(a) Cirujano(a),[GA] Consulta nueva,43.5
5,Médico(a) Cirujano(a),[GA] Consulta nueva+docencia,7.0
6,Médico(a) Cirujano(a),[GA] Endoscopía digestiva,22.0
7,Médico(a) Cirujano(a),[GA] Otras Actividades Clínicas,63.0
8,Médico(a) Cirujano(a),[GA] Procedimientos especialidad,16.5
9,Médico(a) Cirujano(a),[GA] Reunión Clínica,6.0
